# <img style="float: left; padding-right: 20px; width: 200px" src="https://raw.githubusercontent.com/raxlab/imt2200-data/main/media/logo.jpg">  IMT 2200 - Introducción a Ciencia de Datos
**Pontificia Universidad Católica de Chile**<br>
**Instituto de Ingeniería Matemática y Computacional**<br>
**Semestre 2025-S2**<br>
**Profesor:** Rodrigo A. Carrasco <br>

# <h1><center>Actividad 04: Obteniendo Datos de la Web</center></h1>

Esta actividad busca aplicar conocimientos sobre lectura de datos desde la web en distintos formatos (scrapping y APIs) para la creación de un dataset unificado.

## Instrucciones

Este Notebook contiene las instrucciones a realizar para la actividad. 

<b>Al finalizarla, deben subir el Notebook y los archivos generados en un único archivo .zip, al módulo de la Actividad 04 en Canvas. Entregas posteriores al cierre de la actividad serán evaluadas con nota 1.0.</b>

## Actividad

Para esta actividad, queremos analizar la calidad del aire de las ciudades más pobladas del mundo. Para esto, realice los siguientes pasos:

**1. Extraer datos con web scrapping y API**

Vamos a extraer una lista de las ciudades más pobladas desde Wikipedia, específicamente en el siguiente URL:

`URL_1 = https://en.wikipedia.org/wiki/List_of_largest_cities#List`

Por otra parte, usaremos la [Open-Meteo API](https://open-meteo.com/). Este es un servicio open-source de meteorología que nos permitirá obtener datos como las coordenadas de una ciudad y sus parámetros climáticos, como la calidad del aire.

* 1.1 Utilizando las librerías `requests`y `BeautifulSoup`, obtenga todas las filas y columnas de la tabla de Wikipedia de las ciudades más grandes del mundo y genere un DataFrame a partir de ellas. Su DataFrame debe contener como mínimo las siguientes columnas: ciudad, país y población estimada.

* 1.2 Transforme la columna de población en valores numéricos y sólo deje las 20 mayores ciudades.

**2. Llamada a la API**

* 2.1 Ahora, utilizando `requests`, haga un llamado al siguiente URL de la API de Open-Meteo, reemplazando el valor `CIUDAD` con cada uno de los nombres de las ciudades de su DataFrame:

`URL_2 = https://geocoding-api.open-meteo.com/v1/search?name={CIUDAD}&count=1&language=en&format=json`

Haga una copia de su DataFrame anterior. En esta copia, agregue dos columnas nuevas y guarde los valores obtenidos de latitud y longitud (sin modificar el DataFrame original).

* 2.2 Con los datos de las coordenadas, podemos acceder a información sobre la calidad del aire actual disponible con Open-Meteo. Nuevamente, para todas las ciudades, utilice el URL dado para obtener el índices de calidad del aire (usaremos el europeo) y la cantidad de partículas en suspensión.

`URL_3 = https://air-quality-api.open-meteo.com/v1/air-quality?latitude={LAT}&longitude={LON}&current=european_aqi,pm10,pm2_5`

Guarde los valores obtenidos en nuevas columnas del mismo DataFrame.

* 2.3 En la documentación de Open-Meteo ([aquí](https://open-meteo.com/en/docs/air-quality-api)), podemos ver el significado de los valores del índice European AQI. Utilizando la función `aqi2str()` entregada, genere una nueva columna `Air Quality` (string) a partir de los valores que obtuvo mediante la API.

* 2.4 Revise los valores obtenidos. ¿Tienen sentido? Si hay valores que considere inválidos o "outliers" (extremadamente altos), descártelos del dataset.

**3. Visualización**

Vamos a generar dos visualizaciones a partir de las ciudades con las que hemos trabajado. Para esto, usaremos una nueva librería de visualización llamada `plotly.express`. Plotly permite generar gráficos interactivos, con tooltips donde podemos mostrar información adicional de nuestro DataFrame, lo cual los hace muy convenientes para la exploración de un dataset.

Lea y complete el código entregado con los valores de su DataFrame. Ejecute las celdas y responda:

* Entre las ciudades más pobladas, ¿cuál es la calidad de aire más común?

* ¿Cómo es la relación entre tamaño de población y calidad del aire de las ciudades?

* ¿Hay algún lugar del mundo donde se vea una mayor concentración de grandes ciudades? Si la hay, ¿cómo es la calidad del aire en estas zonas?

## Rúbrica

- Si han hecho todo y sólo hay errores menores: 7.0
- Si sólo llegaron hasta la parte 2.1: 5.0
- Menos que eso: 1.0

### 0. Algunas librerías

Las siguientes son algunas de las librerías que recomendamos usar para esta Actividad. Puede agregar más si lo requiere.

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

### 1. Extraer datos

#### 1.1 Respuesta:

In [19]:
url_cities = "https://en.wikipedia.org/wiki/List_of_largest_cities"
headers = {"User-Agent": "imt2200-Act-4"}
page = requests.get(url_cities, headers=headers, timeout=10)
soup = bs(page.text, "html.parser")

tablas = soup.find_all('table')
tabla_cities = tablas[1]

df_cities = pd.DataFrame(columns=['city','country','population'])

# Iteramos desde la segunda fila (saltando el header principal)
for row in tabla_cities.find_all('tr')[1:23]:
    ths = row.find_all("th")
    tds = row.find_all("td")
    
    if len(ths) > 0 and len(tds) >= 2:
        city = ths[0].text.strip()
        country = tds[0].text.strip()
        population = tds[1].text.strip()
        
        df_cities = pd.concat([
            df_cities,
            pd.DataFrame({
                'city': [city],
                'country': [country],
                'population': [population]
            })
        ], ignore_index=True)

print(df_cities)



              city        country  population
0            Tokyo          Japan  37,468,000
1            Delhi          India  28,514,000
2         Shanghai          China  25,582,000
3        São Paulo         Brazil  21,650,000
4      Mexico City         Mexico  21,581,000
5            Cairo          Egypt  20,076,000
6           Mumbai          India  19,980,000
7          Beijing          China  19,618,000
8            Dhaka     Bangladesh  19,378,000
9            Osaka          Japan  19,281,000
10   New York City  United States  18,819,000
11          Tehran           Iran  16,896,000
12         Karachi       Pakistan  15,400,000
13         Kolkata          India  15,333,000
14    Buenos Aires      Argentina  14,967,000
15       Chongqing          China  14,838,000
16        Istanbul         Turkey  14,751,000
17          Manila    Philippines  13,482,000
18           Lagos        Nigeria  13,463,000
19  Rio de Janeiro         Brazil  13,293,000
20         Tianjin          China 

#### 1.2 Respuesta:

In [31]:
df_cities['population'] = df_cities['population'].str.replace(r",", "", regex=True)
df_cities['population'] = df_cities['population'].astype(int)
df_cities.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   city        21 non-null     object
 1   country     21 non-null     object
 2   population  21 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 636.0+ bytes


### 2. Uso de API

#### 2.1 Respuesta:

In [ ]:
url_API = 'https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1&language=en&format=json'
headers = {"User-Agent": "imt2200-Act-4"}

new_data = df_cities

def get_coordinates(city):
    r = requests.get(url_API.format(city=city)).json()
    if "results" in r:
        lat = r["results"][0]["latitude"]
        lon = r["results"][0]["longitude"]
        return lat, lon
    else:
        return None, None
    

new_data["lat"], new_data["lon"] = zip(*df_cities["city"].apply(get_coordinates))
new_data.drop(columns=['latitud', 'longitud'])

,city,country,population,latitud,longitud,lat,lon
0,Tokyo,Japan,37468000,23.7104,90.40744,35.68950,139.69171
1,Delhi,India,28514000,23.7104,90.40744,28.65195,77.23149
2,Shanghai,China,25582000,23.7104,90.40744,31.22222,121.45806
3,São Paulo,Brazil,21650000,23.7104,90.40744,-23.54750,-46.63611
4,Mexico City,Mexico,21581000,23.7104,90.40744,19.42847,-99.12766
5,Cairo,Egypt,20076000,23.7104,90.40744,30.06263,31.24967
6,Mumbai,India,19980000,23.7104,90.40744,19.07283,72.88261
7,Beijing,China,19618000,23.7104,90.40744,39.90750,116.39723
8,Dhaka,Bangladesh,19378000,23.7104,90.40744,23.71040,90.40744
9,Osaka,Japan,19281000,23.7104,90.40744,34.69379,135.50107


In [ ]:
new_data

,city,country,population,lat,lon
0,Tokyo,Japan,37468000,35.68950,139.69171
1,Delhi,India,28514000,28.65195,77.23149
2,Shanghai,China,25582000,31.22222,121.45806
3,São Paulo,Brazil,21650000,-23.54750,-46.63611
4,Mexico City,Mexico,21581000,19.42847,-99.12766
5,Cairo,Egypt,20076000,30.06263,31.24967
6,Mumbai,India,19980000,19.07283,72.88261
7,Beijing,China,19618000,39.90750,116.39723
8,Dhaka,Bangladesh,19378000,23.71040,90.40744
9,Osaka,Japan,19281000,34.69379,135.50107


#### 2.2 Respuesta:

In [78]:
def get_air(lat: float, lon: float):
    URL_3 = f'https://air-quality-api.open-meteo.com/v1/air-quality?latitude={lat}&longitude={lon}&current=european_aqi,pm10,pm2_5'
    r = requests.get(URL_3).json()
    data = r.get("current", {})
    return {
        "aqi": data.get("european_aqi"),
        "pm10": data.get("pm10"),
        "pm2_5": data.get("pm2_5")
    }

resultados = []

for i in range(len(new_data)):
    datos = get_air(lat=new_data['lat'][i], lon=new_data['lon'][i])
    resultados.append(datos)

df_air = pd.DataFrame(resultados)
new_data = pd.concat([new_data, df_air], axis=1)

In [77]:
new_data

,city,country,population,lat,lon,aqi,pm10,pm2_5
0,Tokyo,Japan,37468000,35.68950,139.69171,32,32.7,31.2
1,Delhi,India,28514000,28.65195,77.23149,90,76.1,61.2
2,Shanghai,China,25582000,31.22222,121.45806,49,17.6,16.4
3,São Paulo,Brazil,21650000,-23.54750,-46.63611,67,24.9,24.6
4,Mexico City,Mexico,21581000,19.42847,-99.12766,42,12.7,12.4
5,Cairo,Egypt,20076000,30.06263,31.24967,53,27.4,15.9
6,Mumbai,India,19980000,19.07283,72.88261,39,36.0,18.0
7,Beijing,China,19618000,39.90750,116.39723,91,57.9,57.0
8,Dhaka,Bangladesh,19378000,23.71040,90.40744,73,54.0,53.4
9,Osaka,Japan,19281000,34.69379,135.50107,27,11.0,10.6


#### 2.3 Respuesta:

In [ ]:
# ==== CODIGO ENTREGADO - NO MODIFICAR ====
air_quality = {
    "Good": [0, 20],
    "Fair": [20, 40],
    "Moderate": [40, 60],
    "Poor": [60, 80],
    "Very Poor": [80, 100],
    "Extremely Poor": [100, float('inf')]
}

def aqi2str(aqi):
    for key, (low, high) in air_quality.items():
        if low <= aqi < high:
            return key
    return "Unknown"

new_data["aqi"] = new_data["aqi"].apply(lambda x: x.iloc[0])
print(new_data['aqi'])
new_data['Air Quality'] = new_data['aqi'].apply(aqi2str)



    aqi  aqi
0    32   32
1    32   32
2    32   32
3    32   32
4    32   32
5    32   32
6    32   32
7    32   32
8    32   32
9    32   32
10   32   32
11   32   32
12   32   32
13   32   32
14   32   32
15   32   32
16   32   32
17   32   32
18   32   32
19   32   32
20   32   32


C:\Users\User\AppData\Local\Temp\ipykernel_12080\702473150.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_data["aqi"] = new_data["aqi"].apply(lambda x: x.iloc[0])


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
# 2.3

#### 2.4 Respuesta:

In [ ]:
# 2.4

### 3. Visualizar datos

* ¿Cómo son los valores de calidad de aire para las ciudades más pobladas? ¿Cuál es lo más común?

* ¿Cómo es la relación entre tamaño de población y calidad del aire de una ciudad?

* ¿Hay algún lugar del mundo donde se vea una mayor concentración de grandes ciudades? Si la hay, ¿cómo es la calidad del aire?

In [ ]:
# Figura 1: Barplot de calidad del aire
import plotly.express as px

by_quality = new_df.groupby('Air Quality').size().reset_index(name='Count')

fig = px.bar(by_quality,
            x='Air Quality',
            y='Count',
            title="Calidad del aire de 80 ciudades más pobladas del mundo",
            labels={
                "Count": "Cantidad de ciudades",
                "Air Quality": "Calidad del aire"
            },
            color='Air Quality')

fig.update_layout(
    height=400,
    width=900,
)
fig.update_xaxes(categoryorder='array',
                 categoryarray= ["Good", "Fair", "Moderate", "Poor", "Very Poor", "Extremely Poor"]
)
fig.show()

#### Respuesta:

In [ ]:
# Figura 2: Scattermap entre población y calidad del aire

fig = px.scatter(df_filtered,
                 x="Population",
                 y="eu_aqi",
                 title="Relación entre población y calidad del aire",
                 labels={
                     "Population": "Población",
                     "eu_aqi": "Calidad del aire (EU AQI)"
                    },
                    hover_data={
                        "City": True,
                        "Country": True,
                        "Population": True,
                        "eu_aqi": True,
                        "Air Quality": True
                        }
                )

fig.update_layout(
    height=500,
    width=900,
)

#### Respuesta:

In [ ]:
# Figura 3: Mapa mundial de ciudades más pobladas

fig = px.scatter_geo(data_frame=df_filtered, # Su dataframe
                    lat='lat', # Columna de latitud
                    lon='lon', # Columna de longitud
                    color='eu_aqi', # Columna que representa el color de los puntos
                    hover_name='City', # Columna para el titulo del tooltip
                    projection="natural earth",
                    color_continuous_scale=px.colors.sequential.Inferno_r,
                    title="Calidad del aire de ciudades más pobladas", # Titulo del grafico
                    hover_data={
                        # Qué columnas mostrar en el tooltip
                        "Country": True,
                        "eu_aqi": True,
                        "Air Quality": True
                        # Puede agregar otras...
                    },
                )

fig.update_layout(
    margin={"r":0,"t":50,"l":0,"b":0}, # Márgenes del gráfico
    height=600, # Altura del gráfico
    width=800, # Ancho del gráfico
)
fig.update_traces(
    marker=dict(size=10), # Tamaño de los puntos
)
fig.show()

#### Respuesta: